# Project Stage 3: Entity Matching#

The entity we performed our match was on a set of restaurants in New York City. The data for the tables was taken from two different web sources, namely, Tripadvisor and Yelp.

We used the py_entitymatching package to help in this process.

In [241]:
# Import py_entitymatching package
import py_entitymatching as em
import os
import pandas as pd

## Reading in Input Tables##

We read the CSV files and set 'ID' as the key attribute.

Input table A corresponds to data from TripAdvisor, and input table B corresponds to data from Yelp.

In [242]:
path_A = 'C:\\Users\\bharg\\Documents\\TripAdvisor_Restaurants.csv'
path_B = 'C:\\Users\\bharg\\Documents\\Yelp_Restaurants.csv'
#path_A = 'C:\\Users\\Aribhit\\TripAdvisor_Restaurants.csv'
#path_B = 'C:\\Users\\Aribhit\\Yelp_Restaurants.csv'

In [243]:
A = em.read_csv_metadata(path_A, key='Id', encoding = 'cp1252')

Metadata file is not present in the given path; proceeding to read the csv file.


In [244]:
B = em.read_csv_metadata(path_B, key='Id', encoding = 'cp1252')

Metadata file is not present in the given path; proceeding to read the csv file.


# Data Pre-processing#

Since there was a lot of variance in how *Address* attribute was defined, created a new attribute called *Street* that is extracted from *Address* (by considering the part of the string till the state appears).

Also converted every string type attribute to lower case.

In [245]:
A['Street'] = A.apply(lambda row : row['Address'][0:row['Address'].find('New')],axis=1)
A['Name']=A['Name'].str.lower()
A['Street']=A['Street'].str.lower()
A['Address']=A['Address'].str.lower()

In [246]:
def cleaning(row) :
    for string in ['New','Jersey','NY','NJ']:
        index = row['Address'].find(string)
        if index == -1 :
            continue
        return row['Address'][0:index]
    return row['Address']
    
B['Street'] = B.apply(cleaning,axis =1)
B['Name']=B['Name'].str.lower()
B['Street']=B['Street'].str.lower()
B['Address']=B['Address'].str.lower()

Deleting *Phone* attribute in case it might act as an unique ID and game the system

In [247]:
#A1 = A.copy(deep=True)
#B1 = B.copy(deep=True)
#del A['Phone']
#del B['Phone']

## Applying the Blocker##

We have used the combination of two blockers: 
One blocks based on *Name* of the restaurant (Jaccard Measure with 3 grams with a constraint 0.3) and *Street* (Jaccard Measure with 3 grams with a constraint 0.3). 
Next blocker is only on the *Street* attribute(Jaccard Measure with 3 grams with a constraint 0.6).

We use these blockers and then combine the results of two different blockers using union for the following reasons.
*Street* (from *Address*) only because it can capture some pairs where names are same but differ by a new word. Ex. (alfa ristorante, alfa). The constraint is higher - 0.6
*Name* only to capture restaurants that have similar names (constraint is 0.3). But added *Street* based rule on top of that to eliminate chain restaurants with multiple branches at different locations (eg: Shake shacks at Manhattan, Shake Shacks at Brooklyn). The constraint is lower threshold in this case compared to the earlier blocker.

First, get all possible features for blocking.

In [248]:
block_f = em.get_features_for_blocking(A, B, validate_inferred_attr_types=False)

First rule-based blocker uses *Name* and *Street* attributes

In [249]:
rb = em.RuleBasedBlocker()
ab = em.AttrEquivalenceBlocker()
rb.add_rule(['Name_Name_jac_qgm_3_qgm_3(ltuple, rtuple) < 0.5'], block_f)
rb.add_rule(['Street_Street_jac_qgm_3_qgm_3(ltuple, rtuple) < 0.3'], block_f)

'_rule_1'

In [250]:
C = rb.block_tables(A, B, l_output_attrs=['Name', 'Street', 'Address','Cuisines','Take Out','Phone',
                                          'Saturday Opening time','Saturday Closing time','Sunday Opening time',
                                          'Sunday Closing time'], 
                    r_output_attrs=['Name', 'Street', 'Address','Cuisines','Take Out','Phone',
                                          'Saturday Opening time','Saturday Closing time','Sunday Opening time',
                                          'Sunday Closing time'], show_progress=True)

0% [##############################] 100% | ETA: 00:00:00

Finding pairs with missing value...



Total time elapsed: 00:00:01
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Second rule-based blocker uses only *Street* attribute

In [251]:
rb2 = em.RuleBasedBlocker()
rb2.add_rule(['Street_Street_jac_qgm_3_qgm_3(ltuple, rtuple) < 0.6'], block_f)
E = rb2.block_tables(A, B, l_output_attrs=['Name', 'Street', 'Address','Cuisines','Take Out','Phone',
                                          'Saturday Opening time','Saturday Closing time','Sunday Opening time',
                                          'Sunday Closing time'], 
                     r_output_attrs=['Name', 'Street', 'Address','Cuisines','Take Out','Phone',
                                          'Saturday Opening time','Saturday Closing time','Sunday Opening time',
                                          'Sunday Closing time'], n_jobs=-1,show_progress=True)

Finding pairs with missing value...


Combining blocker1 and blocker2 results to get candidate set C (which is named F in our code).

In [252]:
F = em.combine_blocker_outputs_via_union([C, E])

Running debugger to see if F is good. 41/50 outputs of debugger are bad matches.Therefore we are proceeding with the above 
blocker

In [14]:
dbg = em.debug_blocker(F, A, B, output_size=50)
dbg.head()

,_id,ltable_Id,rtable_Id,ltable_Name,ltable_Address,ltable_Cuisines,ltable_Saturday Opening time,ltable_Saturday Closing time,ltable_Sunday Opening time,ltable_Sunday Closing time,...,ltable_Street,rtable_Name,rtable_Address,rtable_Cuisines,rtable_Saturday Opening time,rtable_Saturday Closing time,rtable_Sunday Opening time,rtable_Sunday Closing time,rtable_Take Out,rtable_Street
0,0,1308,1183,razza pizza artigianale,"275 grove st jersey city, nj 07302-3601",Italian;Pizza;Vegetarian Friendly,05:30 PM,10:30 PM,NaN,NaN,...,"275 grove st jersey city, nj 07302-360",razza pizza artigianale,275 grove stjersey city nj 07302,Italian;Salad,05:30 PM,10:00 PM,CLOSED,CLOSED,Yes,275 grove st
1,1,2076,2436,sandro's,"306 e 81st st new york city, ny 10028-3984",Italian,05:00 PM,11:00 PM,04:30 PM,10:00 PM,...,306 e 81st st,antonucci cafe,170 e 81st stnew york ny 10028,Italian,12:00 PM,11:00 PM,12:00 PM,10:00 PM,Yes,170 e 81st st
2,2,1974,3094,strip house,"13 e 12th st new york city, ny 10003-4404",American;Steakhouse;Gluten Free Options,05:00 PM,11:30 PM,05:00 PM,10:00 PM,...,13 e 12th st,strip house speakeasy,11 e 12th stnew york ny 10003,Steakhouses,05:00 PM,11:30 PM,CLOSED,CLOSED,No,11 e 12th st
3,3,3174,2890,porter house,"10 columbus cir ste 4 new york city, ny 10019-1210",American;Steakhouse;Grill,11:30 AM,11:00 PM,11:00 AM,10:00 PM,...,10 columbus cir ste 4,porter house bar and grill,10 columbus cirnew york ny 10019,Steakhouses;American (New);Cocktail Bars,11:30 AM,11:00 PM,11:30 AM,10:00 PM,No,10 columbus cir
4,4,2322,2861,salon de ning rooftop,"700 5th avenue at 55th street new york city, ny 10019",American;Bar,05:00 PM,01:00 AM,05:00 PM,12:00 AM,...,700 5th avenue at 55th street,salon de ning,700 5th avethe peninsula hotelnew york ny 10019,Lounges;Cocktail Bars,05:00 PM,01:00 AM,04:00 PM,12:00 AM,No,700 5th avethe peninsula hotel


In [253]:
F.to_csv("F.csv",index=False,encoding = 'cp1252')

In [254]:
F.shape

(1625, 23)

In [255]:
F.head()

,_id,ltable_Id,rtable_Id,ltable_Name,ltable_Street,ltable_Address,ltable_Cuisines,ltable_Take Out,ltable_Phone,ltable_Saturday Opening time,...,rtable_Name,rtable_Street,rtable_Address,rtable_Cuisines,rtable_Take Out,rtable_Phone,rtable_Saturday Opening time,rtable_Saturday Closing time,rtable_Sunday Opening time,rtable_Sunday Closing time
0,0,6,1428,shanghai asian manor,21 mott st,"21 mott st new york city, ny 10013-5032",Chinese;Asian;Vegetarian Friendly,Yes,+1 212-766-6311,11:00 AM,...,shanghai asian manor,21 mott st,21 mott stnew york ny 10013,Shanghainese,Yes,(212) 766-6311,11:00 AM,10:00 PM,11:00 AM,09:00 PM
1,1,11,1489,shake shack,366 columbus ave,"366 columbus ave new york city, ny 10024-5109",American;Fast Food,Yes,+1 646-747-8770,10:30 AM,...,shake shack,366 columbus ave,366 columbus avenew york ny 10024,Burgers;American (New),Yes,(646) 747-8770,10:30 AM,11:00 PM,10:30 AM,11:00 PM
2,2,15,606,blue maiz,606 8th ave,"606 8th ave new york city, ny 10018-3008",Mexican;Latin;Fast Food,Yes,+1 212-704-2106,11:00 AM,...,blue maiz,606 8th ave,606 8th avenew york ny 10018,Mexican,Yes,(212) 704-2106,11:00 AM,10:00 PM,11:00 AM,09:00 PM
3,3,21,1019,dutch fred's,307 w 47th st,"307 w 47th st new york city, ny 10036-2403",American;Bar;Pub,No,+1 646-918-6923,NaN,...,dutch freds,307 w 47th st,307 w 47th stnew york ny 10036,Cocktail Bars;Pubs;Venues & Event Spaces,No,(646) 918-6923,11:00 AM,04:00 AM,11:00 AM,04:00 AM
4,4,26,1972,pret a manger,30 rockefeller plz,"30 rockefeller plz new york city, ny 10112-0015",American;Cafe;Fast Food,Yes,+1 212-246-6944,NaN,...,bar sixtyfive,30 rockefeller plzfl 65,30 rockefeller plzfl 65new york ny 10112,Cocktail Bars;Venues & Event Spaces;American (New),No,(212) 632-5000,CLOSED,CLOSED,04:00 PM,09:00 PM


Taking a sample of 600 tuples from the output, and then we label this sample manually.

In [16]:
S = em.sample_table(F, 600)
S.to_csv('Sample.csv',encoding = 'cp1252')

## Reading the Labelled Sample##
Loading the labeled data table, which is present in a file called 'Labelled_Sample_v2.csv'

In [256]:
L = em.read_csv_metadata("Labelled_Sample_v2.csv", key='_id', encoding = 'cp1252',\
                         ltable=A, rtable=B,fk_ltable='ltable_Id', fk_rtable='rtable_Id')

Metadata file is not present in the given path; proceeding to read the csv file.


Deleting *Phone* attribute again, because it can help determine matches trivially.

In [14]:
del L['ltable_Phone']
del L['rtable_Phone']

## Splitting the Labelled Set##

Splitting the labelled set into training and test set, by putting half the tuple pairs in each.<br>
The development set is called I<br>
The evaluation set is called J

In [257]:
IJ = em.split_train_test(L, train_proportion=0.5, random_state=0)
I = IJ['train']
J = IJ['test']
I.to_csv('I.csv',encoding = 'cp1252')
J.to_csv('J.csv',encoding = 'cp1252')

In [258]:
J.head()

,_id,ltable_Id,rtable_Id,ltable_Name,ltable_Street,ltable_Address,ltable_Phone,ltable_Cuisines,ltable_Take Out,rtable_Name,rtable_Street,rtable_Address,rtable_Phone,rtable_Cuisines,rtable_Take Out,label
434,1164,2882,3032,iridium jazz club,1650 broadway,"1650 broadway new york city, ny 10019-6833",+1 212-582-2121,American;Bar;Pub,No,the iridium,1650 broadway,1650 broadwaynew york ny 10019,(347) 815-8649,Jazz & Blues;American (New),No,1
122,319,778,2666,mykonos blue restaurant & rooftop,127 w 28th st,"127 w 28th st new york city, ny 10001-6102",+1 646-484-4339,Mediterranean;Greek;Seafood,No,mykonos blue,127 w 28th st,127 w 28th stnew york ny 10001,(347) 979-0451,Greek,Yes,1
224,603,1522,599,noodle village,13 mott st,"13 mott st new york city, ny 10013-5025",+1 212-233-0788,Chinese;Asian;Vegetarian Friendly,Yes,noodle village,13 mott st,13 mott stnew york ny 10013,(212) 233-0788,Chinese;Noodles,Yes,1
479,1290,3111,1845,parker and quinn,63 w 39th st,"63 w 39th st new york city, ny 10018-3801",+1 212-729-0277,American;Bar;Vegetarian Friendly,Yes,parker & quinn,64 w 39th st,64 w 39th stnew york ny 10018,(212) 729-0277,American (New);Bars;Breakfast & Brunch,Yes,1
205,540,1350,1949,moustache,90 bedford st,"90 bedford st new york city, ny 10014-3764",+1 212-229-2220,Lebanese;Mediterranean;Middle Eastern,Yes,the little owl,90 bedford st,90 bedford stnew york ny 10014,(212) 741-4695,Desserts;Wine Bars;Mediterranean,Yes,0


## Creating ML-matchers##

Initiating 6 different classifiers (Decision Tree, Random Forest, SVM, Naive Bayes, Logistic Regression, Linear Regression) and then, cross validating them on I set.

In [259]:
dt = em.DTMatcher(name='DecisionTree', random_state=0)
rf = em.RFMatcher(name='RF', random_state=0)
svm = em.SVMMatcher(name='SVM', random_state=0)
nb = em.NBMatcher(name ='NaiveBayes')
lg = em.LogRegMatcher(name='LogReg', random_state=0)
ln = em.LinRegMatcher(name='LinReg')

## Selecting Best Matcher ##

First, we obtain all the features we could use for matching. Ft is our feature table

In [260]:
Ft = em.get_features_for_matching(A, B, validate_inferred_attr_types=False)

Use the system to generate feature vectors from set I. This is called set H

In [261]:
H = em.extract_feature_vecs(I, 
                            feature_table=Ft, 
                            attrs_after='label',
                            show_progress=False)

Perform matches and display results below (after performing cross-validation)

In [262]:
H = em.impute_table(H, 
                exclude_attrs=['_id', 'ltable_Id', 'rtable_Id', 'label'],
                strategy='mean')

In [161]:
result = em.select_matcher([dt, rf, svm, ln, lg,nb], table=H, 
        exclude_attrs=['_id', 'ltable_Id', 'rtable_Id', 'label'],
        k=5,
        target_attr='label', metric_to_select_matcher='f1', random_state=0)
result['cv_stats']

,Matcher,Average precision,Average recall,Average f1
0,DecisionTree,0.966529,0.969566,0.967772
1,RF,0.965328,0.969566,0.967215
2,SVM,0.827648,0.995455,0.902929
3,LinReg,0.969032,0.954895,0.961627
4,LogReg,0.976138,0.969460,0.972647
5,NaiveBayes,0.975238,0.974112,0.974515


Picking Random Forest as it has the highest average F1 score. We are not adding any rule based matchers as the precision,recall and F1 scores are already above the required thresholds.

## Evaluating Best Matcher##

As we picked Random Forest as the best matcher, now we apply it on the evaluation set (set J; defined earlier) to find how well it performs.

Create a new Random Forest matcher and train it on set H (feature table obtained from set I):

In [263]:
rf = em.RFMatcher(name='RF', random_state=0)

In [264]:
rf.fit(table=H, 
       exclude_attrs=['_id', 'ltable_Id', 'rtable_Id', 'label'], 
       target_attr='label')

In [265]:
Test_Ft = em.read_csv_metadata("F.csv", key='_id', encoding = 'cp1252',\
                         ltable=A, rtable=B,fk_ltable='ltable_Id', fk_rtable='rtable_Id')

Metadata file is not present in the given path; proceeding to read the csv file.


Extracting features from set J:

In [266]:
Test_Ft = em.extract_feature_vecs(Test_Ft, feature_table=Ft,
                             show_progress=False)

In [267]:
Test_Ft = Test_Ft.dropna(axis =0,how ='any')
Test_Ft.to_csv("Test_Ft.csv",index=False)

In [268]:
Test_Ft = em.read_csv_metadata("Test_Ft.csv", key='_id', encoding = 'cp1252',\
                         ltable=A, rtable=B,fk_ltable='ltable_Id', fk_rtable='rtable_Id')

Metadata file is not present in the given path; proceeding to read the csv file.


In [269]:
Test_Ft.head()

,_id,ltable_Id,rtable_Id,Id_Id_exm,Id_Id_anm,Id_Id_lev_dist,Id_Id_lev_sim,Name_Name_jac_qgm_3_qgm_3,Name_Name_cos_dlm_dc0_dlm_dc0,Name_Name_jac_dlm_dc0_dlm_dc0,...,Take_Out_Take_Out_exm,Take_Out_Take_Out_jac_qgm_3_qgm_3,Street_Street_jac_qgm_3_qgm_3,Street_Street_cos_dlm_dc0_dlm_dc0,Street_Street_jac_dlm_dc0_dlm_dc0,Street_Street_mel,Street_Street_lev_dist,Street_Street_lev_sim,Street_Street_nmw,Street_Street_sw
0,0,6,1428,0,0.004202,4,0.00,1.0,1.0,1.0,...,1,1.0,0.666667,1.0,1.000000,0.981818,1,0.909091,9.0,10.0
1,1,11,1489,0,0.007388,3,0.25,1.0,1.0,1.0,...,1,1.0,0.761905,1.0,1.000000,0.988235,1,0.941176,15.0,16.0
2,2,15,606,0,0.024752,3,0.00,1.0,1.0,1.0,...,1,1.0,0.687500,1.0,1.000000,0.983333,1,0.916667,10.0,11.0
3,8,29,1231,0,0.023558,3,0.25,1.0,1.0,1.0,...,1,1.0,0.705882,1.0,1.000000,0.984615,1,0.923077,11.0,12.0
4,9,32,2719,0,0.011769,4,0.00,1.0,1.0,1.0,...,1,1.0,0.451613,0.5,0.333333,0.870813,6,0.727273,13.0,16.0


Computing predictions on set J:

In [270]:
predictions = rf.predict(table=Test_Ft, exclude_attrs=['_id', 'ltable_Id', 'rtable_Id'], 
              append=True, target_attr='predicted', inplace=False, return_probs=True,
                        probs_attr='proba')

In [271]:
predictions[['_id', 'ltable_Id', 'rtable_Id', 'predicted', 'proba']]

,_id,ltable_Id,rtable_Id,predicted,proba
0,0,6,1428,1,1.0
1,1,11,1489,1,1.0
2,2,15,606,1,1.0
3,8,29,1231,1,1.0
4,9,32,2719,1,0.9
5,10,33,1589,1,0.9
6,12,48,2167,1,0.9
7,13,51,2397,1,1.0
8,14,55,2505,1,1.0
9,15,56,507,1,1.0


In [272]:
ids = predictions['_id'][predictions['predicted']==1]

In [273]:
ids = ids.tolist()

In [274]:
F = em.read_csv_metadata("F.csv", key='_id', encoding = 'cp1252',\
                         ltable=A, rtable=B,fk_ltable='ltable_Id', fk_rtable='rtable_Id')

Metadata file is not present in the given path; proceeding to read the csv file.


In [275]:
F.shape

(1625, 23)

In [276]:
F = F[F['_id'].isin(ids)]

In [277]:
F.shape

(786, 23)

In [278]:
F.to_csv("project4.csv",index =False)

In [298]:
F.head(200)

,ltable_Name,ltable_Street,ltable_Address,ltable_Cuisines,ltable_Take Out,ltable_Phone,ltable_Saturday Opening time,ltable_Saturday Closing time,ltable_Sunday Opening time,ltable_Sunday Closing time,rtable_Name,rtable_Street,rtable_Address,rtable_Cuisines,rtable_Take Out,rtable_Phone,rtable_Saturday Opening time,rtable_Saturday Closing time,rtable_Sunday Opening time,rtable_Sunday Closing time
0,shanghai asian manor,21 mott st,"21 mott st new york city, ny 10013-5032",Chinese;Asian;Vegetarian Friendly,Yes,+1 212-766-6311,11:00 AM,10:00 PM,11:00 AM,09:00 PM,shanghai asian manor,21 mott st,21 mott stnew york ny 10013,Shanghainese,Yes,(212) 766-6311,11:00 AM,10:00 PM,11:00 AM,09:00 PM
1,shake shack,366 columbus ave,"366 columbus ave new york city, ny 10024-5109",American;Fast Food,Yes,+1 646-747-8770,10:30 AM,11:00 PM,10:30 AM,11:00 PM,shake shack,366 columbus ave,366 columbus avenew york ny 10024,Burgers;American (New),Yes,(646) 747-8770,10:30 AM,11:00 PM,10:30 AM,11:00 PM
2,blue maiz,606 8th ave,"606 8th ave new york city, ny 10018-3008",Mexican;Latin;Fast Food,Yes,+1 212-704-2106,11:00 AM,10:00 PM,11:00 AM,09:00 PM,blue maiz,606 8th ave,606 8th avenew york ny 10018,Mexican,Yes,(212) 704-2106,11:00 AM,10:00 PM,11:00 AM,09:00 PM
8,barn joo 35,34 w 35th st,"34 w 35th st new york city, ny 10001-2256",Asian;Korean;Gastropub,Yes,+1 212-564-4430,12:00 PM,02:00 AM,12:00 PM,11:00 PM,barn joo 35,34 w 35th st,34 w 35th stnew york ny 10001,Korean;Gastropubs;Tapas/Small Plates,Yes,(212) 564-4430,12:00 PM,01:00 AM,12:00 PM,11:00 PM
9,southgate bar & restaurant,154 central park s,"154 central park s new york city, ny 10019-1510",American;Gluten Free Options,No,+1 212-484-5120,07:00 AM,10:30 PM,07:00 AM,10:30 PM,southgate bar & restaurant,160 central park south,160 central park southnew york ny 10019,Breakfast & Brunch;American (Traditional),No,(210) 491-5845,07:00 AM,01:00 AM,07:00 AM,01:00 AM
10,friedman's,132 w 31st st,"132 w 31st st new york city, ny 10001-3406",American;Vegetarian Friendly;Vegan Options,Yes,+1 212-971-9400,08:30 AM,10:00 PM,08:30 AM,09:00 PM,friedman’s,132 w 31st st,132 w 31st stnew york ny 10001,American (New);Gluten-Free;Breakfast & Brunch,Yes,(212) 971-9400,08:30 AM,10:00 PM,08:30 AM,09:00 PM
12,boulud sud,20 w 64th st,"20 w 64th st new york city, ny 10023-7129",Mediterranean;Vegetarian Friendly;Vegan Options,No,+1 212-595-1313,05:00 PM,11:30 PM11:00 AM,05:00 PM,11:00 PM11:00 AM,boulud sud,20 west 64th street,20 west 64th streetnew york ny 10023,Mediterranean,No,(212) 595-1313,11:00 AM,11:00 PM,11:00 AM,10:00 PM
13,rosa mexicano - union square,9 e 18th st,"9 e 18th st new york city, ny 10003-1903",Mexican;Latin;Central American,Yes,+1 212-533-3350,11:30 AM,11:30 PM,11:30 AM,04:00 PM,rosa mexicano,9 e 18th st,9 e 18th stnew york ny 10003,Mexican,Yes,(212) 533-3350,11:30 AM,11:30 PM,11:30 AM,10:30 PM
14,benoit new york,60 w 55th st,"60 w 55th st new york city, ny 10019-5308",French;Contemporary;Vegetarian Friendly,No,+1 646-943-7373,11:45 AM,11:00 PM,11:45 AM,11:00 PM,benoit,60 w 55th st,60 w 55th stnew york ny 10019,French;Wine Bars,Yes,(646) 943-7373,11:45 AM,11:00 PM,11:45 AM,11:00 PM
15,the original soupman,259a w 55th st,"259a w 55th st new york city, ny 10019-5202",Fast Food;Soups;American,Yes,+1 212-956-0900,11:00 AM,08:00 PM,11:00 AM,07:00 PM,the original soupman,259a w 55th st,259a w 55th stnew york ny 10019,Soup;Seafood,Yes,(212) 956-0900,11:00 AM,08:00 PM,11:00 AM,07:00 PM


In [ ]:
del F['_id']
del F['ltable_Id']
del F['rtable_Id']

# Project Stage 4 - Data Merging
Please find the code for the Data Merging part of Stage 4 below.

In [280]:
p4 = pd.DataFrame(columns=['Name', 'Address','Cuisines','Take Out','Phone',
                                          'Saturday Opening time','Saturday Closing time','Sunday Opening time',
                                          'Sunday Closing time'])

In [309]:

def name_merger(row) :
    return row['ltable_Name'][:1].upper()+row['ltable_Name'][1:] if len(row['ltable_Name']) >= len(row['rtable_Name'])\
    else row['rtable_Name'][:1].upper()+row['rtable_Name'][1:] 

def address_merger(row) :
    return row['ltable_Address'].title() if len(row['ltable_Address']) >= len(row['rtable_Address']) \
    else row['rtable_Address'].title()

def cuisine_merger(row):
    result = set()
    union1 = row['ltable_Cuisines'].split(';')
    union1 = [x.lower() for x in union1]
    union2 = row['rtable_Cuisines'].split(';')
    union2 = [x.lower() for x in union2]
    if len(union1)>=len(union2):
        result.update(union1)
    else:
        result.update(union2)
    return "; ".join([x.title() for x in list(result)])

def takeout_merger(row) :
    if row['ltable_Take Out'] == 'Yes' or row['rtable_Take Out'] == 'Yes' :
        return 'Yes'
    else :
        return 'No'

def phone_merger(row) :
    return row['ltable_Phone'] if len(row['ltable_Phone']) >= len(row['rtable_Phone']) \
    else row['rtable_Phone']    
    
def sat_open_merger(row) :
    if row['ltable_Saturday Opening time'].lower() == 'nan' :
        return row['rtable_Saturday Opening time']
    if row['rtable_Saturday Opening time'].lower() == 'nan' :
        return row['ltable_Saturday Opening time']
    if row['rtable_Saturday Opening time'].lower() == 'closed' or row['ltable_Saturday Opening time'].lower() == 'closed':
        return 'Closed'
    return row['ltable_Saturday Opening time'] if len(row['ltable_Saturday Opening time']) <= len(row['rtable_Saturday Opening time']) \
    else row['rtable_Saturday Opening time']  

def sat_close_merger(row) :
    if row['ltable_Saturday Closing time'].lower() == 'nan' :
        return row['rtable_Saturday Closing time']
    if row['rtable_Saturday Closing time'].lower() == 'nan' :
        return row['ltable_Saturday Closing time']
    if row['rtable_Saturday Closing time'].lower() == 'closed' or row['ltable_Saturday Closing time'].lower() == 'closed':
        return 'Closed'
    return row['ltable_Saturday Closing time'] if len(row['ltable_Saturday Closing time']) <= len(row['rtable_Saturday Closing time']) \
    else row['rtable_Saturday Closing time']  

def sun_open_merger(row) :
    if row['ltable_Sunday Opening time'].lower() == 'nan' :
        return row['rtable_Sunday Opening time']
    if row['rtable_Sunday Opening time'].lower() == 'nan' :
        return row['ltable_Sunday Opening time']
    if row['rtable_Sunday Opening time'].lower() == 'closed' or row['ltable_Sunday Opening time'].lower() == 'closed':
        return 'Closed'
    return row['ltable_Sunday Opening time'] if len(row['ltable_Sunday Opening time']) <= len(row['rtable_Sunday Opening time']) \
    else row['rtable_Sunday Opening time']  
    
def sun_close_merger(row):
    if row['ltable_Sunday Closing time'].lower() == 'nan' :
        return row['rtable_Sunday Closing time']
    if row['rtable_Sunday Closing time'].lower() == 'nan' :
        return row['ltable_Sunday Closing time']
    if row['rtable_Sunday Closing time'].lower() == 'closed' or row['ltable_Sunday Closing time'].lower() == 'closed':
        return 'Closed'
    return row['ltable_Sunday Closing time'] if len(row['ltable_Sunday Closing time']) <= len(row['rtable_Sunday Closing time']) \
    else row['rtable_Sunday Closing time']  

In [310]:
p4['Name'] = F.apply(name_merger,axis =1)
p4['Address'] = F.apply(address_merger,axis =1)
p4['Cuisines'] = F.apply(cuisine_merger,axis =1)
p4['Take Out'] = F.apply(takeout_merger,axis =1)
p4['Phone'] = F.apply(phone_merger,axis =1)
p4['Saturday Opening time'] = F.apply(sat_open_merger,axis =1)
p4['Saturday Closing time'] = F.apply(sat_close_merger,axis =1)
p4['Sunday Opening time'] = F.apply(sun_open_merger,axis =1)
p4['Sunday Closing time'] = F.apply(sun_close_merger,axis =1)

In [311]:
p4.head(100)

,Name,Address,Cuisines,Take Out,Phone,Saturday Opening time,Saturday Closing time,Sunday Opening time,Sunday Closing time
0,Shanghai asian manor,"21 Mott St New York City, Ny 10013-5032",Chinese; Vegetarian Friendly; Asian,Yes,+1 212-766-6311,11:00 AM,10:00 PM,11:00 AM,09:00 PM
1,Shake shack,"366 Columbus Ave New York City, Ny 10024-5109",American; Fast Food,Yes,+1 646-747-8770,10:30 AM,11:00 PM,10:30 AM,11:00 PM
2,Blue maiz,"606 8Th Ave New York City, Ny 10018-3008",Fast Food; Mexican; Latin,Yes,+1 212-704-2106,11:00 AM,10:00 PM,11:00 AM,09:00 PM
8,Barn joo 35,"34 W 35Th St New York City, Ny 10001-2256",Korean; Gastropub; Asian,Yes,+1 212-564-4430,12:00 PM,02:00 AM,12:00 PM,11:00 PM
9,Southgate bar & restaurant,"154 Central Park S New York City, Ny 10019-1510",Gluten Free Options; American,No,+1 212-484-5120,07:00 AM,10:30 PM,07:00 AM,10:30 PM
10,Friedman's,"132 W 31St St New York City, Ny 10001-3406",Vegan Options; Vegetarian Friendly; American,Yes,+1 212-971-9400,08:30 AM,10:00 PM,08:30 AM,09:00 PM
12,Boulud sud,"20 W 64Th St New York City, Ny 10023-7129",Vegan Options; Vegetarian Friendly; Mediterranean,No,+1 212-595-1313,05:00 PM,11:00 PM,05:00 PM,10:00 PM
13,Rosa mexicano - union square,"9 E 18Th St New York City, Ny 10003-1903",Central American; Mexican; Latin,Yes,+1 212-533-3350,11:30 AM,11:30 PM,11:30 AM,04:00 PM
14,Benoit new york,"60 W 55Th St New York City, Ny 10019-5308",Vegetarian Friendly; Contemporary; French,Yes,+1 646-943-7373,11:45 AM,11:00 PM,11:45 AM,11:00 PM
15,The original soupman,"259A W 55Th St New York City, Ny 10019-5202",Soups; Fast Food; American,Yes,+1 212-956-0900,11:00 AM,08:00 PM,11:00 AM,07:00 PM


In [312]:
p4.to_csv("E.csv",index = False,encoding = 'cp1252')